In [63]:
import numpy as np
from qiskit import *
from random import random, randint
from qiskit.circuit import Parameter
import matplotlib.pyplot as plt

# import feature map
from feature_mapping_initialize_method import *

In [71]:
# Preparing feature-mapped circuit
# Inputs: 
#   num_qubits: number of qubits (log_2 of number of features), 
#   feature_data: data pertaining to each feature
# Outputs: 
#   qc: prepared quantum circuit
def prepare_circuit(num_qubits, feature_data):
    feature_data = normalize_data(feature_data)
    q = QuantumRegister(num_qubits)
    qc = QuantumCircuit(q)
    qc.initialize(feature_data, qc.qubits)
    return qc

In [79]:
# Measures a quantum circuit and processes the output
# Inputs: 
#   circuit: quantum circuit
#   s: number of shots
# Outputs:
#   counts: dictionary of results from measuring the circuit
def process_output(circuit, s):
    
    arr = []

    simulator = Aer.get_backend('aer_simulator')

    job = execute(circuit, simulator, shots=s)
    result = job.result()
    counts = result.get_counts(circuit)
    return counts

# Classifies counts based on the parity of the bits
# Inputs:
#   counts: dictionary containing counts from the circuit
#   shots: number of shots used in measurement
# Outputs:
#   dictionary containing the probability distribution of the data falling into each class
def parity_probabilities(counts, shots=100):
    class_0 = 0
    class_1 = 0

    for keyBefore, value in counts.items():
        key = keyBefore.split()[0]
        temp = int(key[0]) ^ int(key[1])
        for i in range(2, len(key)):
            temp = temp ^ int(key[i])
        
        if temp == 0:
            class_0 += value
        else:
            class_1 += value
    return {0: class_0 / shots, 1: class_1 / shots}

# Performs a variational quantum circuit with arbitrary rotation gates
# Inputs:
#   register: quantum register for the qubits
#   angles: list of parameters to be applied with the rotation gates
#   qc: prepared quantum circuit
# Outputs:
#   parity_processed:
def rotation(num_qubits, angles, qc):

    measurement = ClassicalRegister(num_qubits)
    qc.add_register(measurement)
    for i in range(num_qubits):
        qc.ry(angles[i], i)
    
    for i in range(num_qubits - 1, 0, -1):
        qc.cx(i, i - 1)

    qc.cx(0, num_qubits - 1)

    qc.measure_all()
    s = 100
    shots = process_output(qc, 100)
    
    parity_processed = parity_probabilities(shots)
    return parity_processed

q = QuantumRegister(4)
initial_angles = [2 * np.pi * random() for _ in range(4)]
len(q)

4

In [33]:
def mse(shots_count, expected_value):
    probability_of_expected = shots_count[expected_value]
    return (1 - probability_of_expected) ** 2

# y_values are either 0, 1
# received_measurements are the dictionary of probabilities
def count_costs(x_values, y_values, angles, received_measurements):
    cost = 0

    for i in range(len(x_values)):
        cost += mse(received_measurements[i], y_values[i])
    
    return cost / len(x_values)
    

In [81]:
def cost_across_x(num_qubits, x, y, angles):
    measured = []
    for i in x:
        qc = prepare_circuit(num_qubits, i)
        measured.append(rotation(num_qubits, angles, qc))
    cost_count = count_costs(x, y, angles, measured)
    return cost_count

def optimize_angles(angles, cost, x_values, y_values, num_qubits, delta=0.1):
    new_angles = []
    new_cost = cost

    for i in range(len(angles)):
        step_up = [j for j in angles]
        step_up[i] += delta

        step_down = [j for j in angles]
        step_down[i] -= delta

        cost_up = cost_across_x(num_qubits, x_values, y_values, step_up)
        cost_down = cost_across_x(num_qubits, x_values, y_values, step_down)
        
        if cost_up < new_cost:
            new_cost = cost_up
            new_angles = [angle for angle in step_up]
        if cost_down < new_cost:
            new_cost = cost_down
            new_angles = [angle for angle in step_down]

    return new_angles, new_cost
        

In [85]:
# random samples, toy data
entries = 32

x = []
for _ in range(entries):
    temp = [20 * random() for _ in range(4)]
    while temp in x:
        temp = 20 * random()
    x.append(temp)

y = [randint(0, 1) for _ in range(entries)]

measured = []

num_qubits = int(np.log2(len(x[0])))

initial_angles = [2 * np.pi * random() for _ in range(num_qubits)]

for i in range(len(x)):
    qc = prepare_circuit(num_qubits, x[i])
    measured.append(rotation(num_qubits, initial_angles, qc))
    
cost = count_costs(x, y, initial_angles, measured)
print(cost)

optimize_angles(initial_angles, cost, x, y, num_qubits)

0.34344375000000005


([3.8484110388520985, 1.4562631110686006], 0.3367125)